# 1 - Install missing dependencies


In [0]:
!apt-get -qq install libspatialindex-dev
!pip install -q earthpy

# 2 - Authenticate to Google Drive 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# 3 - Import necessary python libraries and mount google drive

In [0]:
import numpy as np
import rasterio as rio
from rasterio.windows import Window
import earthpy.plot as ep

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 4 - Read the data

We read all bands into a dictionary, that hold the band names as key and the respective array as value. All data is cut to the same extent of 3000 x 3000 pixels.


In [0]:
list_of_layers = [
    'VV_sigma0', 
    'VV_gamma0', 
    'VV_gamma0flat',
    'layover', 'shadow'
    ]

# this refers to the version we produced during data generation, i.e. volume model and buffer
data_version = 'SRTMGL1_003_volume_buf_0'
buf_version = 'SRTMGL1_003_volume_buf_30'

# we read all layers into a dictionary 
dataDict = {}
for layer in list_of_layers:
    with rio.open('/content/drive/My Drive/slope_correction {}/{}.tif'.format(data_version, layer)) as src:
        print('Reading ' + layer)
        dataDict[layer] = np.nan_to_num(src.read(window=Window(0, 340, 3000, 3000)))[0]
        print(dataDict[layer].shape)
        
# we read all layers into a dictionary for 30m buffered
dataDict_buf = {}
for layer in ['layover', 'shadow']:
    with rio.open('/content/drive/My Drive/slope_correction {}/{}.tif'.format(buf_version, layer)) as src:
        print('Reading buffered' + layer)
        dataDict_buf[layer] = np.nan_to_num(src.read(window=Window(0, 340, 3000, 3000)))[0]
        print(dataDict_buf[layer].shape)

buf_version = 'SRTMGL1_003_volume_buf_50'
# we read all layers into a dictionary for 50m buffered
dataDict_buf3 = {}
for layer in ['layover', 'shadow']:
    with rio.open('/content/drive/My Drive/slope_correction {}/{}.tif'.format(buf_version, layer)) as src:
        print('Reading buffered' + layer)
        dataDict_buf3[layer] = np.nan_to_num(src.read(window=Window(0, 340, 3000, 3000)))[0]
        print(dataDict_buf3[layer].shape)


# we read all layers into a dictionary for 100m buffered
buf_version = 'SRTMGL1_003_volume_buf_100'
dataDict_buf5 = {}
for layer in ['layover', 'shadow']:
    with rio.open('/content/drive/My Drive/slope_correction {}/{}.tif'.format(buf_version, layer)) as src:
        print('Reading buffered' + layer)
        dataDict_buf5[layer] = np.nan_to_num(src.read(window=Window(0, 340, 3000, 3000)))[0]
        print(dataDict_buf5[layer].shape)

# 5 - Prepare data for plotting

We create two 3-D array for the RGB plotting of the Sentinel-1 data. Furthermore, the layover and shadow masks are inverted for plotting.


In [0]:
# create layover/shadow masked dicts (i.e. one with normal LS, and one with buffered LS)
ls_masked = {}
for layer in list_of_layers:
    
    ls_masked[layer] = dataDict[layer].astype('float32').copy()
    # set all layover and shadow areas to nan (for image plots)
    ls_masked[layer][dataDict['layover'] == 0] = np.nan

# subsets for zoom ins
#alpha_rRad_10m = dataDict['alpha_rRad'][-1100:-100, 600:1600].copy()
gamma0 = dataDict['VV_gamma0'][-1100:-100, 600:1600].copy()
gamma0flat = dataDict['VV_gamma0flat'][-1100:-100, 600:1600].copy()
gamma0flat[gamma0flat == 0] = np.nan

# subset layover and shadows and invert mask for plotting
layover = dataDict['layover'][-1100:-100, 600:1600].astype('float32').copy()
layover[layover == 1] = np.nan
layover[layover == 0] = 1

shadow = dataDict['shadow'][-1100:-100, 600:1600].astype('float32').copy()
shadow[shadow == 1] = np.nan
shadow[shadow == 0] = 1

layover_buf = dataDict_buf['layover'][-1100:-100, 600:1600].astype('float32').copy()
layover_buf[layover_buf == 1] = np.nan
layover_buf[layover_buf == 0] = 1

shadow_buf = dataDict_buf['shadow'][-1100:-100, 600:1600].astype('float32').copy()
shadow_buf[shadow_buf == 1] = np.nan
shadow_buf[shadow_buf == 0] = 1

layover_buf3 = dataDict_buf3['layover'][-1100:-100, 600:1600].astype('float32').copy()
layover_buf3[layover_buf3 == 1] = np.nan
layover_buf3[layover_buf3 == 0] = 1

shadow_buf3 = dataDict_buf3['shadow'][-1100:-100, 600:1600].astype('float32').copy()
shadow_buf3[shadow_buf3 == 1] = np.nan
shadow_buf3[shadow_buf3 == 0] = 1

layover_buf5 = dataDict_buf5['layover'][-1100:-100, 600:1600].astype('float32').copy()
layover_buf5[layover_buf5 == 1] = np.nan
layover_buf5[layover_buf5 == 0] = 1

shadow_buf5 = dataDict_buf5['shadow'][-1100:-100, 600:1600].astype('float32').copy()
shadow_buf5[shadow_buf5 == 1] = np.nan
shadow_buf5[shadow_buf5 == 0] = 1

# 6 - Plot figure

In [0]:
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(18, 10))

# force nn interploation method
interpolation_method = 'nearest'

# extent
x = [600, 1600, 1600, 600, 600]
y = [2900, 2900, 1900, 1900, 2900]

ax1 = ax[0][0]
img1 = ax1.imshow(dataDict['VV_gamma0'], cmap=plt.cm.Greys_r)
img1.set_clim(vmin=-30, vmax=15)
ax1.set_title(r'VV backscatter ($\gamma^0$)')
ax1.set_axis_off()
ax1.plot(x, y, 'r--')
cbar1 = fig.colorbar(img1, ax=ax1) 
cbar1.set_label('dB')

ax2 = ax[1][0]
img2 = ax2.imshow(dataDict['VV_gamma0'][-1100:-100, 600:1600], cmap=plt.cm.Greys_r)
img2.set_clim(vmin=-30, vmax=15)
ax2.set_title(r'VV backscatter ($\gamma^0$)')
ax2.set_axis_off()
cbar2 = fig.colorbar(img2, ax=ax2) 
cbar2.set_label('dB')

ax3 = ax[0][1]
ax3.scatter(0, 0, cmap=plt.cm.bwr)
img3 = ax3.imshow(dataDict['VV_gamma0flat'][-1100:-100, 600:1600], cmap=plt.cm.Greys_r)
img3.set_clim(vmin=-30, vmax=15)
cbar3 = fig.colorbar(img3, ax=ax3) 
cbar3.set_label('dB')
ax3.set_title(r'$\gamma^0_f$ - VV & Lay/Sha mask (without buffer)')
ax3.set_axis_off()

img3 = ax3.imshow(layover, cmap=plt.cm.bwr_r, interpolation=interpolation_method)
img3.set_clim(vmin=1, vmax=1)

img3 = ax3.imshow(shadow, cmap=plt.cm.bwr, interpolation=interpolation_method)
img3.set_clim(vmin=1, vmax=1)
ep.draw_legend(im_ax=img3, bbox=(0,1,0,0),  titles=['Shadow', 'Layover'], cmap=plt.cm.bwr, classes=[0, 1])

ax4 = ax[1][1]
img4 = ax4.imshow(dataDict['VV_gamma0flat'][-1100:-100, 600:1600], cmap=plt.cm.Greys_r)
img4.set_clim(vmin=-30, vmax=15)
cbar4 = fig.colorbar(img4, ax=ax4) 
cbar4.set_label('dB')
ax4.set_title(r'$\gamma^0_f$ - VV & Lay/Sha mask (buffer = 30m)')
img4 = ax4.imshow(layover_buf, cmap=plt.cm.bwr_r, interpolation=interpolation_method)
img4.set_clim(vmin=1, vmax=1)
img4 = ax4.imshow(shadow_buf, cmap=plt.cm.bwr, interpolation=interpolation_method)
img4.set_clim(vmin=1, vmax=1)
ep.draw_legend(im_ax=img4, bbox=(0,1,0,0),  titles=['Shadow', 'Layover'], cmap=plt.cm.bwr, classes=[0, 1])
ax4.set_axis_off()

ax5 = ax[0][2]
ax5.scatter(0, 0, cmap=plt.cm.bwr)
img5 = ax5.imshow(dataDict['VV_gamma0flat'][-1100:-100, 600:1600], cmap=plt.cm.Greys_r)
img5.set_clim(vmin=-30, vmax=15)
cbar5 = fig.colorbar(img5, ax=ax5) 
cbar5.set_label('dB')
ax5.set_title(r'$\gamma^0_f$ - VV & Lay/Sha mask (buffer = 50m)')
ax5.set_axis_off()

img5 = ax5.imshow(layover_buf3, cmap=plt.cm.bwr_r, interpolation=interpolation_method)
img5.set_clim(vmin=1, vmax=1)

img5 = ax5.imshow(shadow_buf3, cmap=plt.cm.bwr, interpolation=interpolation_method)
img5.set_clim(vmin=1, vmax=1)
ep.draw_legend(im_ax=img5, bbox=(0,1,0,0),  titles=['Shadow', 'Layover'], cmap=plt.cm.bwr, classes=[0, 1])

ax6 = ax[1][2]
ax6.scatter(0, 0, cmap=plt.cm.bwr)
img6 = ax6.imshow(dataDict['VV_gamma0flat'][-1100:-100, 600:1600], cmap=plt.cm.Greys_r)
img6.set_clim(vmin=-30, vmax=15)
cbar6 = fig.colorbar(img6, ax=ax6) 
cbar6.set_label('dB')
ax6.set_title(r'$\gamma^0_f$ - VV & Lay/Sha mask (buffer = 100m)')
ax6.set_axis_off()

img6 = ax6.imshow(layover_buf5, cmap=plt.cm.bwr_r, interpolation=interpolation_method)
img6.set_clim(vmin=1, vmax=1)

img6 = ax6.imshow(shadow_buf5, cmap=plt.cm.bwr, interpolation=interpolation_method)
img6.set_clim(vmin=1, vmax=1)
ep.draw_legend(im_ax=img6, bbox=(0,1,0,0),  titles=['Shadow', 'Layover'], cmap=plt.cm.bwr, classes=[0, 1])


plt.tight_layout()
plt.savefig('/content/drive/My Drive/slope_correction/Figure_6.png')